In [37]:
import pandas as pd
from datetime import datetime as dt
import plotly.express as px
import numpy as np

In [9]:
df=pd.read_csv('data/cohort.csv', parse_dates=['acquisition_month','cancelled_month'])

In [10]:
df.head()

,customer_id,acquisition_month,cancelled_month,sex
0,0001146a-c9ad-464f-8644-4228d8a5fd5e,2020-01-01,2020-02-01,female
1,0003c75d-54a2-4939-8ae3-09f55cf3a18c,2020-01-01,2020-02-01,female
2,00057ffc-f5cb-4ba6-a6a2-3f9e95d19306,2020-01-01,2020-02-01,female
3,000dbc0c-3f8e-45c0-bfba-dbce6a529499,2020-01-01,2020-02-01,female
4,00118706-f2e7-4ff9-83bc-4c654c21eb58,2020-01-01,2020-02-01,female


In [12]:
data_bar

,acquisition_month,sex,customer_id,cancelled_month
0,2020-01-01,female,4981,3434
1,2020-01-01,male,19,16
2,2020-02-01,female,4483,2883
3,2020-02-01,male,17,11
4,2020-03-01,female,7557,3325
5,2020-03-01,male,43,19
6,2020-04-01,female,2886,1990
7,2020-04-01,male,14,8
8,2020-05-01,female,7954,5561
9,2020-05-01,male,46,39


In [38]:
data_bar=df.groupby(['acquisition_month']).count().reset_index()
fig = px.bar(data_bar, x="acquisition_month", y="customer_id", 
             title="Acquisition by month", 
                labels={
                     "customer_id": "Number of New Subs",
                     "acquisition_month": "Acquisition Month",
                     "sex":"Gender"
                 })

fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})


fig.show()


In [63]:
df['month_churned']=(df['cancelled_month'] - df['acquisition_month'])//np.timedelta64(1, 'M')
df.head()

,customer_id,acquisition_month,cancelled_month,sex,month_churned
0,0001146a-c9ad-464f-8644-4228d8a5fd5e,2020-01-01,2020-02-01,female,1.0
1,0003c75d-54a2-4939-8ae3-09f55cf3a18c,2020-01-01,2020-02-01,female,1.0
2,00057ffc-f5cb-4ba6-a6a2-3f9e95d19306,2020-01-01,2020-02-01,female,1.0
3,000dbc0c-3f8e-45c0-bfba-dbce6a529499,2020-01-01,2020-02-01,female,1.0
4,00118706-f2e7-4ff9-83bc-4c654c21eb58,2020-01-01,2020-02-01,female,1.0


In [54]:
data_line.dtypes

acquisition_month    datetime64[ns]
month_churned               float64
customer_id                   int64
dtype: object

In [71]:
data_line=df.groupby(['acquisition_month','month_churned'])['customer_id'].count().reset_index()
data_line['cumulative_churn']=data_line.groupby(['acquisition_month'])['customer_id'].cumsum().reset_index()['customer_id']
data_line['total_churn_per_cohort']=data_line.groupby(['acquisition_month'])['cumulative_churn'].transform(max)
data_line.head()

,acquisition_month,month_churned,customer_id,cumulative_churn,total_churn_per_cohort
0,2020-01-01,1.0,1860,1860,3450
1,2020-01-01,2.0,270,2130,3450
2,2020-01-01,3.0,10,2140,3450
3,2020-01-01,4.0,50,2190,3450
4,2020-01-01,5.0,115,2305,3450


In [78]:
data_line['Churn_Rate']=data_line['cumulative_churn']/data_line['total_churn_per_cohort']
data_line['Retention_Rate']=1-data_line['Churn_Rate']
data_line['acquisition_month_s'] = data_line['acquisition_month'].astype(str)

In [79]:
data_line.head()

,acquisition_month,month_churned,customer_id,cumulative_churn,total_churn_per_cohort,Churn_Rate,Retention_Rate,acquisition_month_s
0,2020-01-01,1.0,1860,1860,3450,0.539130,0.460870,2020-01-01
1,2020-01-01,2.0,270,2130,3450,0.617391,0.382609,2020-01-01
2,2020-01-01,3.0,10,2140,3450,0.620290,0.379710,2020-01-01
3,2020-01-01,4.0,50,2190,3450,0.634783,0.365217,2020-01-01
4,2020-01-01,5.0,115,2305,3450,0.668116,0.331884,2020-01-01


In [84]:

fig = px.line(data_line, x="month_churned", y="Retention_Rate", 
              color='acquisition_month_s',
              title="Cohorts' Stay Rate by Acquisition Month", 
              labels={
                     "month_churned": "Number of Months",
                     "acquisition_month_s": "Acquisition Month",
                     "Retention_Rate":"Retention Rate"})
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

fig.show()

In [ ]:
def clv(retention_rate,discount_rate,margin, cost, month_churned):
    clv=sum(margin*retention_rate/(1+discount_rate)^month_chruned)-cost
    return clv
    